# Examples usage of the majority voting I/O processor

This notebook should eventually turn into documentation.

In [ ]:
import aconfig

from granite_io.io.base import ChatCompletionInputs
from granite_io.io.granite_3_2.granite_3_2 import Granite3Point2InputOutputProcessor
from granite_io.backend.openai import OpenAIBackend
from granite_io.io.voting import MajorityVotingProcessor, integer_normalizer

In [ ]:
# Connect to a backend running on localhost.
port = 11434
url = f"http://localhost:{port}/v1"
model_name = "granite3.2:2b"
backend = OpenAIBackend(
    aconfig.Config(
        {"openai_base_url": url, "model_name": model_name}, override_env_vars=False
    ),
)

In [ ]:
# Ask a question with an integral answer
completion_inputs = ChatCompletionInputs(
    messages=[
        {
            "role": "user",
            "content": "What is 234651 + 13425?\nAnswer with just a number please.",
        }
    ],
    thinking=True,
    num_return_sequences=1,
)
completion_inputs

In [ ]:
# Run the question through the base model
base_processor = Granite3Point2InputOutputProcessor(backend=backend)
results = base_processor.create_chat_completion(completion_inputs)
for r in results.results:
    print(r.next_message.content)

In [ ]:
# Wrap the base model's I/O processor in a majority voting I/O processor.
voting_processor = MajorityVotingProcessor(
    base_processor, integer_normalizer, samples_per_completion=100
)
results = voting_processor.create_chat_completion(completion_inputs)
for r in results.results:
    print(r.next_message.content)

In [ ]:
# What's the actual answer?
234651 + 13425